# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '2e4008cc-066c-48f7-b933-de50c0dade99', 'created_at': '2025-08-14T00:03:12.094680+00:00', 'updated_at': '2025-08-14T00:03:12.094680+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
2e4008cc-066c-48f7-b933-de50c0dade99


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [6]:
input = { "messages": "한국 반도체 현황. 모두 한글로 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업 현황 - 한국 반도체 산업의 전반적인 상태와 2023년 현재의 현황을 파악합니다. 생산량 및 수출 관련 최신 통계를 바탕으로 현재 산업의 상황을 분석하고, 주요 수출 국가에 대한 정보와 추가적인 시장 동향을 제시합니다. 
3. 한국 반도체 기업 순위 - 2023년 기준 주요 반도체 기업인 삼성전자와 SK하이닉스를 포함한 한국 반도체 기업들의 시장 내 순위를 조사합니다. 이들 기업의 글로벌 반도체 시장에서의 위치를 분석하고, 다른 주요 기업들과의 비교를 통해 경쟁력을 평가합니다. 
4. 한국 반도체 수출 국가별 통계 - 한국 반도체 산업의 주요 수출 대상 국가와 각 국가별 수출량 및 비중을 통계적으로 분석하여, 한국 반도체의 국제적 위치를 확인합니다. 
5. 반도체 기술 발전 동향 - 한국 반도체 기술의 최근 발전 동향과 미래 전망을 소개합니다. 미래 반도체 기술 발전에 있어 어떤 혁신이 일어날지, 그리고 한국이 이러한 기술 발전에서 어떤 위치를 차지하고 있는지를 분석합니다. 
6. 한국 반도체 산업 지원 정책 - 한국 정부의 반도체 산업 지원 정책과 프로그램을 소개하고, 이러한 정책들이 한국 반도체 산업에 미치는 영향을 분석합니다. 
7. 글로벌 반도체 시장과 한국의 역할 - 글로벌 반도체 시장, 특히 미국과 중국 간의 반도체 경쟁 속에서 한국 반도체 산업의 역할을 평가합니다. 더불어, 한국 반도체 산업이 향후 시장에서 어떻게 위치될 가능성이 있는지를 전망합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  

In [7]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [8]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업 현황 - 한국 반도체 산업의 전반적인 상태와 2023년 현재의 현황을 파악합니다. 생산량 및 수출 관련 최신 통계를 바탕으로 현재 산업의 상황을 분석하고, 주요 수출 국가에 대한 정보와 추가적인 시장 동향을 제시합니다. 
3. 한국 반도체 기업 순위 - 2023년 기준 주요 반도체 기업인 삼성전자와 SK하이닉스를 포함한 한국 반도체 기업들의 시장 내 순위를 조사합니다. 이들 기업의 글로벌 반도체 시장에서의 위치를 분석하고, 다른 주요 기업들과의 비교를 통해 경쟁력을 평가합니다. 
4. 한국 반도체 수출 국가별 통계 - 한국 반도체 산업의 주요 수출 대상 국가와 각 국가별 수출량 및 비중을 통계적으로 분석하여, 한국 반도체의 국제적 위치를 확인합니다. 
5. 반도체 기술 발전 동향 - 한국 반도체 기술의 최근 발전 동향과 미래 전망을 소개합니다. 미래 반도체 기술 발전에 있어 어떤 혁신이 일어날지, 그리고 한국이 이러한 기술 발전에서 어떤 위치를 차지하고 있는지를 분석합니다. 
6. 한국 반도체 산업 지원 정책 - 한국 정부의 반도체 산업 지원 정책과 프로그램을 소개하고, 이러한 정책들이 한국 반도체 산업에 미치는 영향을 분석합니다. 
7. 글로벌 반도체 시장과 한국의 역할 - 글로벌 반도체 시장, 특히 미국과 중국 간의 반도체 경쟁 속에서 한국 반도체 산업의 역할을 평가합니다. 더불어, 한국 반도체 산업이 향후 시장에서 어떻게 위치될 가능성이 있는지를 전망합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  

In [9]:
len(state["values"]["slides"])

8

In [10]:
state["values"]["slides"]

[{'description': '한국 반도체 현황에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'slide_type': 'title',
  'topic': '한국 반도체 현황',
  'idx': 0,
  'name': '타이틀',
  'content': "## 한국 반도체 현황에 대한 프레젠테이션 타이틀 페이지\n\n### Introduction\n한국 반도체 산업은 지난 수십 년 동안 세계 시장에서 중추적인 위치를 차지해 왔습니다. 특히 2023년 현재, 한국은 글로벌 반도체 산업에서 13%의 시장 점유율을 차지하며, 중국과 미국에 이어 세계 3위의 반도체 제조 강국으로 자리 잡고 있습니다 [1][2]. 이러한 입지는 메모리 반도체 분야에서 두각을 나타낸 삼성전자와 SK하이닉스와 같은 주요 기업들에 의해서 견고히 유지되고 있습니다 [3].\n\n### Current Status of South Korea's Semiconductor Industry in 2023\n2023년은 한국 반도체 산업에 도전적인 해로 남을 것입니다. 한국은행이 발표한 보고서에 따르면, 지난 해 전 세계 반도체 시장의 경기 침체로 인해 수요가 급감하면서 메모리 반도체 가격이 대폭 하락했고, 이에 따라 한국의 반도체 수출액도 큰 폭으로 감소했습니다 [4]. 실제로, 2023년 한국의 반도체 수출액은 최근 5년 동안 가장 낮은 약 1,310억 9천만 달러를 기록했습니다 [5]. 그러나, 한국의 경제는 여전히 글로벌 반도체 시장 내에서 강력한 입지를 유지하고 있습니다 [6].\n\n### Future Prospects\n미래를 내다보면, 한국 반도체 산업의 성장 잠재력은 여전히 매우 큽니다. 한국은 메모리 반도체 분야에서는 여전히 세계시장을 지배하는 위치에 있으며, 특히 HBM(고대역폭 메모리) 시장에서는 세계 시장 점유율의 약 95%를 차지하고 있다는 점이 이를 뒷받침합니다 [3

In [11]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])